In [1]:
import pandas as pd
import numpy as np
import os
import umap
import json

import networkx as nx
from gensim.models import KeyedVectors
from nodevectors import Node2Vec, GGVec

import plotly.express as px

In [2]:
# Shape (2931112, 23)
users  = pd.read_csv('SocialTalk 2023/accounts.csv', on_bad_lines='skip', low_memory=False) #We are encoutering some bad lines creating some error so we skip over them
users = users[:10000]
users['label'] = users['Id'].apply(lambda x: f'Account:{x}') #adding label here
users['Social network'] = users['Social network'].replace(['None'],np.nan) # replacing 'None' in Social network with np.nan

In [3]:
# Slicing the data frame
clean_df = users[['Id','City id','Gender','Social network','Followers',"Estimated reach","Avg. views per video"]]

In [4]:
clean_df['Id'] = clean_df['Id'].apply(lambda x: f'Account:{x}')
clean_df = clean_df.rename(columns={'City id': 'City','Social network':'Social_network',
                                    "Avg. views per video":"Avg._views_per_video",
                                    "Estimated reach":"Estimated_reach"}).set_index('Id')

#Country-ID mapping
city_ids = pd.read_csv('SocialTalk 2023/cities.csv', low_memory=False) #Shape (7087, 2)

clean_df['City'] = clean_df['City'].map(city_ids.set_index('Id')['Name'].to_dict())
clean_df['City'] = clean_df['City'].str.replace(" ","_") #Replace white space by "_"

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
## Filling NaN values with UKN
clean_df.fillna("UKNOWN",inplace=True)

In [6]:
## Handling Numerical Columns

In [7]:
clean_df["Followers"]=clean_df["Followers"].astype('int64')
for i in clean_df.columns[-3:]:
    if i !='Avg._views_per_video':
        clean_df[i] = pd.qcut(clean_df[i],q=3,labels=["low","average","high"],duplicates='drop')
    else:
        clean_df[i] = pd.qcut(clean_df[i],q=4,labels=["low","average","high"],duplicates='drop')

In [8]:
## creating edges from users data
edges = []
for column in clean_df.columns:
    series_tuples = [
        (account, f'{column}:{value}') for account, value in clean_df[column].items()
        ]
    edges.extend(series_tuples)

In [9]:
## removing all the edges that have value as low (numerical columns)
e2 = [i for i in edges if not i[1].endswith('low')]
len(e2)

48331

In [10]:
graph = pd.DataFrame(e2, columns=['source', 'target'])
graph.to_csv('edge_list.csv',index=False)

In [11]:
G = nx.from_pandas_edgelist(graph)

In [ ]:
g2v = Node2Vec()
n2v_embeddings = g2v.fit(G)
g2v.save_vectors("wheel_model_v2.bin")

Making walks... Done, T=4.18
Mapping Walk Names... Done, T=1.56
Training W2V... WARNING: gensim word2vec version is unoptimizedTry version 3.6 if on windows, versions 3.7 and 3.8 have had issues
Done, T=102.49


In [ ]:
'''
Trying GGvec
'''

'\nTrying GGvec\n'

In [ ]:
"""import csrgraph as cg
import nodevectors

G = cg.read_edgelist("edge_list.csv", directed=False, sep=',')
ggvec_model = nodevectors.GGVec(verbose=True,max_epoch=50,learning_rate=0.01,order=2) # Learning rate 0.05 produces 0.0015 loss
embeddings = ggvec_model.fit_transform(G)
#ggvec_model.save('GGmodel')"""

'import csrgraph as cg\nimport nodevectors\n\nG = cg.read_edgelist("edge_list.csv", directed=False, sep=\',\')\nggvec_model = nodevectors.GGVec(verbose=True,max_epoch=50,learning_rate=0.01,order=2) # Learning rate 0.05 produces 0.0015 loss\nembeddings = ggvec_model.fit_transform(G)\n#ggvec_model.save(\'GGmodel\')'

In [ ]:
g2v.accuracy()

AttributeError: 'Node2Vec' object has no attribute 'accuracy'